# Modelling fluid flows

- Importing modules

In [5]:
!ls

README.md  data  notebook  src


In [4]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
import pysindy as ps
import pydmd as pdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'pysindy'